In [7]:
import os
os.chdir("/home/julian/Uni/MasterThesis/code/experiments")

from masterthesis.data import load_h5ad

In [8]:
# load the python AnnData object
acinar_ann = load_h5ad("/home/julian/Uni/MasterThesis/data/acinar_sce.h5ad")

# TODO: Convert float64 to float32 for handling by scanpy

## Preprocessing

In [9]:
import scanpy as sc
import math

### Filter Cells

In [10]:
# Notdone in psupertime??

### Select highly variable Genes

Adds columns to `.var` (see [documentation](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.highly_variable_genes.html) )

In [15]:
# selected Genes after preprocessing in R
sel_genes = ["REG3A", "AMY2A", "MT2A", "OLFM4",
             "SYCN", "CELA2B", "FGL1", "AMY2B",
             "MT1G", "TM4SF1", "CELA2A", "PDK4", 
             "TACSTD2", "CD44", "PNLIPRP2", "ALB", 
             "ERP27", "LDHA", "REG3G", "CTRL", "CLPS",
             "FOS", "HSPA8", "SERPINA3", "CELA3B", "CRP" ]

In [11]:
sc.pp.highly_variable_genes(acinar_ann, inplace=True)

In [12]:
acinar_copy = acinar_ann[:, acinar_ann.var.highly_variable]

In [13]:
acinar_copy.shape

(411, 2347)

In [16]:
# Selection diverges from R results:
any([gene in acinar_copy.var_names for gene in sel_genes])

False

### Filter Genes

Discard genes only expressed in a small number (1%) of cells

In [21]:
cell_thresh = math.ceil(0.01 * acinar_ann.n_obs)

In [23]:
sc.pp.filter_genes(acinar_copy, min_cells=cell_thresh, inplace=True)

In [25]:
acinar_copy.shape

(411, 651)

### Denoising / Dropout correction

Mean over 10 highest correlated observations / cells

In [18]:
# TODO
pass

### Scaling

Normalization (mean = 0, std = 1)

In [26]:
### Scaling

# Normalization (mean = 0, std = 1)

# TODO
sc.pp.scale(acinar_copy)